In [1]:
!pip install requests-futures
import pandas as pd
akshit_df = './mlchallenge_set_2021.tsv'
akshit_valid = './mlchallenge_set_validation.tsv'
sam_df = './drive/MyDrive/mlchallenge_set_2021_edited.txt'
#sam_valid = './mlchallenge_set_validation.tsv'
#SA_valid=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_validation.tsv',header=None)
#SA_df=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_2021.tsv',header=None)
#df=SA_df
#valid=SA_valid
df = pd.read_table(sam_df)
#valid = pd.read_table(sam_valid,sep='\t')
df.columns=['category','primary_image_url','All Links','Tags','index']
#valid.columns=['ID', 'Group']

In [348]:
import re
from collections import Counter
freq=Counter()
attribute=[['']]*len(df)
trialrange=100000
start = 900000
for x in range(start,start+trialrange):#range(int(len(df)/10)):#len(df)
    attribute[x]=df.iloc[x,3].lower()
    attribute[x] = re.sub(r'[()]','', attribute[x])
    attribute[x] = re.split(r',', attribute[x])
    attribute[x] = [a.split(':') for a in attribute[x]]
    freq+=Counter([i[0] for i in attribute[x]])
    tempdict={}
    for i in attribute[x]:
            try:
                tempdict[i[0]]=float(i[1])
            except:
                try:
                    tempdict[i[0]]=i[1]
                except:
                    pass
    attribute[x]=tempdict

df['seg']=attribute
#print(df['seg'])

In [349]:
for i in df['seg'][start:start+trialrange]:
  if 'color' in i:
    try:
      i['color']=re.split(r'[-\s,/]', i['color'])
    except:
      i.pop('color',None)
    #print(i['color'])

In [5]:
Brands=[]
Images=[]
Colors = []
color_images = []
needs_color = [] # indexes of things that need color
simple_colors = {'black','white','red','blue','green','yellow','brown','purple','pink','gray','grey'}
trialrange = 100000
print(trialrange)
for i in range(trialrange):
    try:
        for word in df['seg'].iloc[i]['color']:
            if word in simple_colors:
              if word == 'gray':
                Colors.append('grey')
              else:
                Colors.append(word)
              color_images.append(df['primary_image_url'].iloc[i])
              break
            needs_color.append(i)
    except:
      needs_color.append(i)
      continue
#print(Colors)

100000


In [6]:
m = 4000
#!pip install requests-futures
from PIL import Image, ImageFile
import requests
import urllib.request
from io import BytesIO
import numpy as np
i = 0
ImageFile.LOAD_TRUNCATED_IMAGES = True
image_array = []
images = []
from concurrent.futures import as_completed
from requests_futures.sessions import FuturesSession
with FuturesSession() as session:
  futures = [session.get(i) for i in color_images[0:m]]
  for future in as_completed(futures):
      if i % 500 == 0:
        print(i)
      resp = future.result()
      img = Image.open(BytesIO(resp.content))
      img = img.convert('RGB')
      img = img.resize((150,150))
      pix=np.asarray(img)
      pix=pix.astype('float32')
      pix/=255.0
      images.append(pix)
      i+=1

0
500
1000
1500
2000
2500
3000
3500


In [ ]:
#@title Old url requests
'''
n = 3000
from PIL import Image, ImageFile
import requests
import urllib.request
from io import BytesIO
import numpy as np

ImageFile.LOAD_TRUNCATED_IMAGES = True
image_array = []
images = []
max_height = 0
max_width = 0
i = 0

def loadImage(URL):
    with urllib.request.urlopen(URL) as url:
        img = Image.open(BytesIO(url.read()))
    return img

for url in color_images[0:n]:
    if i%500 == 0:
      print(i)
    i+=1
    img = loadImage(url)
    img = img.convert('RGB')
    img = img.resize((200,200))
    pix=np.asarray(img)
    pix=pix.astype('float32')
    pix/=255.0
    images.append(pix)
'''

In [ ]:
print(len(images))
print(len(needs_color))

In [7]:
color_dict = {}
num = 0
labels = []
color_nums = []
for b in Colors:
    if b not in color_dict:
        color_nums.append(b)
        color_dict[b] = num
        num+=1
        
    labels.append(color_dict[b])
print(len(labels))
m = 4000
n = round(m*.8)
from collections import Counter
print(Counter(labels[0:m]))

9541
Counter({4: 1322, 1: 838, 0: 525, 3: 487, 5: 334, 2: 176, 6: 149, 7: 75, 8: 50, 9: 44})


In [8]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout

model = models.Sequential()
model.add(layers.Conv2D(8, (3, 3), activation='softmax', input_shape=(150, 150, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(epsilon=.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#@title Example CNN architecture
'''
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='softmax', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(.8))
model.add(layers.Conv2D(8, (3, 3), activation='softmax'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(.8))
model.add(layers.Conv2D(4, (3, 3), activation='softmax'))
model.add(layers.Flatten())


model.compile(optimizer=tf.keras.optimizers.Adam(epsilon=.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_crossentropy'])
'''

In [ ]:
m = m
for i in range(3):
  history = model.fit(np.asarray(images[0:m]), np.asarray(labels[0:m],np.int16), epochs=50, batch_size = 50,validation_split=.7)

In [428]:
needs_color = []
urls = []
start = 990000
test = 10000
for i,x in enumerate(df['seg'][start:start+test]):
    if x == [''] or ('color' not in x.keys()):
        needs_color.append(x)
        urls.append((df['index'].iloc[start+i],df['primary_image_url'].iloc[start+i]))
        continue
    for c in simple_colors:
      if c in x['color']:
        done = True
    if not done:
      needs_color.append(x)
      urls.append((df['index'].iloc[start+i],df['primary_image_url'].iloc[start+i]))
print(len(needs_color))


3982


In [429]:
import csv
n = len(needs_color)
to_predict = []
i = 0

session = FuturesSession()
futures = [session.get(i[1]) for i in urls[0:n]]
for future in as_completed(futures):
    resp = future.result()
    try:
      img = Image.open(BytesIO(resp.content))
      img = img.convert('RGB')
      img = img.resize((150,150))
      pix=np.asarray(img)
      pix=pix.astype('float32')
      pix/=255.0
      to_predict.append((urls[i][0],pix))
    except:
      print(i)
    if i % 500 == 0:
      print(i)
      
    i+=1

0
500
1000
1500
2000
2500
3000
3500


In [430]:
predictions = []
p = [x[1] for x in to_predict]
p = np.asarray(p)
predictions = model.predict(p,batch_size=50)
'''
dec = len(p)//100
for i in range(97):
  predictions.extend(model.predict(p[i*dec:(i+1)*dec]))
predictions.extend(model.predict(p[9*dec:]))
#predictions = model.predict([p[1] for p in to_predict])
'''
'''
for i in range(n):
  print(urls[i])
  print(f'{color_nums[np.argmax(predictions[i])]}  , {max(predictions[i])}\n')
'''
session = []

In [431]:
#print(predictions[0])
print(len(predictions))

3982


In [432]:
pred_df = pd.DataFrame(columns = ['ID','Predictions'])
for i in range(len(predictions)):
  p = []
  for j in range(10):
    p.append((color_nums[j],predictions[i][j]))
  p.sort()
  pred_df.loc[i] = {'ID':urls[i][0],'Predictions':p}

In [433]:
print(pred_df)
pred_df.to_csv('predictions68.csv')

           ID                                        Predictions
0      993799  [(black, 0.34784812), (blue, 0.006962308), (br...
1      993801  [(black, 0.39113843), (blue, 0.00011291207), (...
2      993803  [(black, 0.7826716), (blue, 0.021367105), (bro...
3      993807  [(black, 0.039410893), (blue, 0.334679), (brow...
4      993810  [(black, 0.06826816), (blue, 0.0066030817), (b...
...       ...                                                ...
3977  1003789  [(black, 0.38781092), (blue, 7.961112e-05), (b...
3978  1003790  [(black, 0.62513196), (blue, 0.0009416709), (b...
3979  1003792  [(black, 0.7456414), (blue, 0.046012845), (bro...
3980  1003794  [(black, 0.9741528), (blue, 0.017098723), (bro...
3981  1003798  [(black, 0.367701), (blue, 0.014606892), (brow...

[3982 rows x 2 columns]
